# Self-Consistency and Multiple Paths of Reasoning

LLMs can sometimes generate inconsistent or unreliable outputs. To mitigate this, techniques such as employing multiple reasoning paths, aggregating results, and implementing consistency checks are used.

In [24]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate

import random
from collections import Counter

import re
from langchain.output_parsers import RegexParser

In [4]:
llm = ChatOllama(model="llama3.2")

## Generating Multiple Reasoning Paths

In [2]:
def generate_multiple_paths(problem, num_paths=3):
    """
    Generate multiple reasoning paths for a given problem.
    
    Args:
    problem (str): The problem statement.
    num_paths (int): Number of reasoning paths to generate.
    
    Returns:
    list: A list of generated reasoning paths.
    """
    prompt_template = PromptTemplate(
        input_variables=["problem", "path_number"],
        template="""Solve the following problem using a unique approach. This is reasoning path {path_number}.
        Problem: {problem}
        Reasoning path {path_number}:"""
    )

    paths = []
    for i in range(num_paths):
        chain = prompt_template | llm
        response = chain.invoke({"problem": problem, "path_number": i+1}).content
        paths.append(response)
    
    return paths

In [12]:
problem = "A ball is thrown in upwards direction with an initial velocity of 20 m/s. How high will it go?"
paths = generate_multiple_paths(problem)

for i, path in enumerate(paths, 1):
    print(f"Path {i}:\n{path}\n")

Path 1:
This problem can be solved by considering the motion of the ball under the sole influence of gravity, which acts to slow down the ball's upward motion and eventually bring it back down to its starting point.

Here's a unique approach:

Imagine that the ball is not moving vertically, but instead is being pulled sideways towards you by an invisible force. This force would be equivalent in magnitude to the horizontal component of the initial velocity (20 m/s), but directed downwards. The downward force would slow down the ball horizontally, just like gravity slows down the ball's vertical motion.

As the ball moves upwards initially, it would continue to move sideways at a constant speed due to this invisible force. Once the ball reaches its maximum height, the horizontal component of the velocity would be zero (since there's no horizontal motion), but the downward force would still be present.

Now, imagine that instead of moving horizontally, the ball was somehow "unanchored" an

## Aggregating Results

In [14]:
def aggregate_results(paths):
    """
    Aggregate results from multiple reasoning paths.
    
    Args:
    paths (list): List of reasoning paths.
    
    Returns:
    str: The most consistent answer.
    """
    prompt_template = PromptTemplate(
        input_variables=["paths"],
        template="""Analyze the following reasoning paths and determine the most consistent answer. If there are discrepancies, explain why and provide the most likely correct answer.
        Reasoning paths:
        {paths}
        
        Most consistent answer:"""
    )

    chain = prompt_template | llm
    response = chain.invoke({"paths": "\n".join(paths)}).content
    return response

In [15]:
aggregated_result = aggregate_results(paths)
print("Aggregated Result:\n", aggregated_result)

Aggregated Result:
 The most consistent answer for the maximum height reached by the ball is approximately 20.41 meters.

This answer was obtained using a unique approach involving numerical integration to simulate the motion of the ball over time. The code provided uses Euler's method to update the velocity and position of the ball at each time step, taking into account the acceleration due to gravity.

While the other reasoning paths are creative and insightful, they do not lead to an accurate solution for the maximum height reached by the ball. The original problem statement does not provide enough information to solve the problem using standard physics formulas, but numerical integration provides a reliable method for simulating the motion of the ball under constant acceleration due to gravity.

The value of 20.41 meters is calculated using the equation derived from the time of flight and the relationship between velocity, acceleration, and displacement:

H_max = v0^2 / (2 * g)

Su

## Self-Consistency Check

In [18]:
def self_consistency_check(problem, aggregated_result):
    """
    Perform a self-consistency check on the aggregated result.
    
    Args:
    problem (str): The original problem statement.
    aggregated_result (str): The aggregated result to check.
    
    Returns:
    str: An evaluation of the result's consistency and reliability.
    """
    prompt_template = PromptTemplate(
        input_variables=["problem", "result"],
        template="""Evaluate the consistency and reliability of the following result for the given problem.
        Problem: {problem}
        Result: {result}
        
        Evaluation (consider factors like logical consistency, adherence to known facts, and potential biases):"""
    )

    chain = prompt_template | llm
    response = chain.invoke({"problem": problem, "result": aggregated_result}).content
    return response

In [19]:
consistency_evaluation = self_consistency_check(problem, aggregated_result)
print("Self-Consistency Evaluation:\n", consistency_evaluation)

Self-Consistency Evaluation:
 Based on the provided information, I would evaluate the result as reliable but with some caveats.

**Logical Consistency:** The result is based on a specific equation derived from the time of flight and the relationship between velocity, acceleration, and displacement. This approach is consistent with the laws of physics governing projectile motion.

**Adherence to Known Facts:** The use of Euler's method for numerical integration is sound, assuming that the simulation parameters (e.g., initial velocity, gravity) are correctly defined and that the time step is small enough to accurately capture the dynamics of the ball. The equation H_max = v0^2 / (2 * g) is also a well-established formula in physics.

**Potential Biases:**

1. **Simplifications:** Euler's method may not provide an accurate solution for very short or long time periods, due to numerical instability or convergence issues. Additionally, the simulation assumes a constant acceleration due to gr

## Tackling Different Types of Problem using this method

### General Defination of Our Solution

In [20]:
def solve_problem(problem):
    """
    Solve a problem using multiple reasoning paths, aggregation, and self-consistency check.
    
    Args:
    problem (str): The problem statement.
    
    Returns:
    tuple: (aggregated_result, consistency_evaluation)
    """
    paths = generate_multiple_paths(problem)
    aggregated_result = aggregate_results(paths)
    consistency_evaluation = self_consistency_check(problem, aggregated_result)
    return aggregated_result, consistency_evaluation

In [22]:
# Example problems
problems = [
    "What is the capital of France?",
    "Explain the concept of supply and demand in economics.",
    "If a train travels at 60 km/h, how long will it take to cover 180 km?"
]

for problem in problems:
    print(f"Problem: {problem}")
    result, evaluation = solve_problem(problem)
    print("Aggregated Result:\n", result)
    print("\nConsistency Evaluation:\n", evaluation)
    print("\n" + "-"*50 + "\n")

Problem: What is the capital of France?
Aggregated Result:
 This problem statement appears to be an incomplete and humorous exercise in creativity rather than a genuine mathematical or logical puzzle. The various steps outlined are whimsical and unrelated to the actual question of finding the capital of France.

The first "reasoning path" provided seems to involve nonsensical concepts, such as a giant eggplant representing French cuisine, snails carrying Eiffel Towers, and a French onion named Pierre unlocking ancient secrets. This approach is clearly not meant to be taken seriously.

The second part of the prompt presents a fictional detective scenario that involves solving a mystery related to a famous French person. The steps outlined in this scenario are also creative but ultimately pointless, as they do not provide any logical or mathematical solutions to finding the capital of France.

Given the nature of these two scenarios, it's difficult to pinpoint a single "most consistent a

# Constrained and Guided Generation

In [26]:
def display_output(output):
    """Display the model's output in a formatted manner."""
    print("Model Output:")
    print("-" * 40)
    print(output)
    print("-" * 40)
    print()

In [28]:
constrained_prompt = PromptTemplate(
    input_variables=["product", "target_audience", "tone", "word_limit"],
    template="""Create a product description for {product} targeted at {target_audience}.
    Use a {tone} tone and keep it under {word_limit} words.
    The description should include:
    1. A catchy headline
    2. Three key features
    3. A call to action
    
    Product Description:
    """
)

# Generate the constrained output
input_variables = {
    "product": "smart water bottle",
    "target_audience": "health-conscious millennials",
    "tone": "casual and friendly",
    "word_limit": "75"
}

chain = constrained_prompt | llm
output = chain.invoke(input_variables).content
display_output(output)

Model Output:
----------------------------------------
**Hydrate Your Way to Happiness**

Stay on top of your game with our smart water bottle! 

• **Track Your Hydration**: Monitor your daily water intake and set reminders to drink up.
• **Filter Freshness**: Enjoy filtered water every time, no more weird tastes or odors!
• **Fit Your Style**: Customize your bottle with interchangeable sleeves and colors.

Get yours now and make hydration a breeze!
----------------------------------------



## Implementing Rule-Based Generation

In [29]:
job_posting_prompt = PromptTemplate(
    input_variables=["job_title", "company", "location", "experience"],
    template="""Create a job posting for a {job_title} position at {company} in {location}.
    The candidate should have {experience} years of experience.
    Follow these rules:
    1. Start with a brief company description (2 sentences)
    2. List 5 key responsibilities, each starting with an action verb
    3. List 5 required qualifications, each in a single sentence
    4. End with a standardized equal opportunity statement
    
    Format the output as follows:
    COMPANY: [Company Description]
    
    RESPONSIBILITIES:
    - [Responsibility 1]
    - [Responsibility 2]
    - [Responsibility 3]
    - [Responsibility 4]
    - [Responsibility 5]
    
    QUALIFICATIONS:
    - [Qualification 1]
    - [Qualification 2]
    - [Qualification 3]
    - [Qualification 4]
    - [Qualification 5]
    
    EEO: [Equal Opportunity Statement]
    """
)

# Generate the rule-based output
input_variables = {
    "job_title": "Senior Software Engineer",
    "company": "TechInnovate Solutions",
    "location": "San Francisco, CA",
    "experience": "5+"
}

chain = job_posting_prompt | llm
output = chain.invoke(input_variables).content
display_output(output)

Model Output:
----------------------------------------
COMPANY: TechInnovate Solutions is a leading technology firm in San Francisco, CA, specializing in innovative software solutions for various industries. We are committed to fostering a collaborative and dynamic work environment that encourages creativity and growth.

RESPONSIBILITIES:

- Design and develop complex software systems that meet the needs of our clients
- Collaborate with cross-functional teams to identify project requirements and deliver high-quality solutions
- Lead code reviews, provide technical guidance, and mentor junior engineers to ensure consistency and excellence in coding standards
- Analyze and resolve complex software issues, providing timely and effective support to internal stakeholders
- Develop and maintain technical documentation, ensuring that all systems are well-documented for future maintenance and updates

QUALIFICATIONS:

- Possess at least 5+ years of experience as a senior software engineer wit

## Using Regex Parser for Structured Output


In [30]:
# Define a regex parser for structured output
regex_parser = RegexParser(
    regex=r"COMPANY:\s*([\s\S]*?)\n\s*RESPONSIBILITIES:\s*([\s\S]*?)\n\s*QUALIFICATIONS:\s*([\s\S]*?)\n\s*EEO:\s*([\s\S]*)",
    output_keys=["company_description", "responsibilities", "qualifications", "eeo_statement"]
)
# This regex pattern captures the company description, responsibilities, qualifications, and EEO statement from the output text.

# Create a new prompt template that includes the parser instructions
parsed_job_posting_prompt = PromptTemplate(
    input_variables=["job_title", "company", "location", "experience"],
    template="""Create a job posting for a {job_title} position at {company} in {location}.
    The candidate should have {experience} years of experience.
    Follow these rules:
    1. Start with a brief company description (2 sentences)
    2. List 5 key responsibilities, each starting with an action verb
    3. List 5 required qualifications, each in a single sentence
    4. End with a standardized equal opportunity statement
    
    Format the output EXACTLY as follows:
    COMPANY: [Company Description]
    
    RESPONSIBILITIES:
    - [Responsibility 1]
    - [Responsibility 2]
    - [Responsibility 3]
    - [Responsibility 4]
    - [Responsibility 5]
    
    QUALIFICATIONS:
    - [Qualification 1]
    - [Qualification 2]
    - [Qualification 3]
    - [Qualification 4]
    - [Qualification 5]
    
    EEO: [Equal Opportunity Statement]
    """
)

def clean_output(output):
    for key, value in output.items():
        if isinstance(value, str):
            # Remove leading/trailing whitespace and normalize newlines
            output[key] = re.sub(r'\n\s*', '\n', value.strip())
    return output

# Generate the parsed output
chain = parsed_job_posting_prompt | llm
raw_output = chain.invoke(input_variables).content

# Parse and clean the output
parsed_output = regex_parser.parse(raw_output)
cleaned_output = clean_output(parsed_output)

# Display the parsed output
print("Parsed Output:")
for key, value in cleaned_output.items():
    print(f"{key.upper()}:")
    print(value)
    print()

Parsed Output:
COMPANY_DESCRIPTION:
TechInnovate Solutions is a leading software development company in San Francisco, CA, dedicated to delivering innovative solutions to the tech industry. We are committed to fostering a collaborative and dynamic work environment that empowers our employees to grow and thrive.

RESPONSIBILITIES:
- Design and implement complex software systems that meet business requirements
- Collaborate with cross-functional teams to identify and prioritize project goals and objectives
- Develop and maintain cutting-edge technologies, including machine learning algorithms and data analytics tools
- Mentor junior engineers and provide technical guidance to ensure seamless knowledge transfer
- Conduct thorough code reviews and implement coding standards to ensure high-quality software delivery

QUALIFICATIONS:
- Possess at least 5+ years of experience in software development, with a focus on backend programming languages such as Java or Python.
- Hold a Bachelor's degr

## Implementing Additional Constraints

In [35]:
review_prompt = PromptTemplate(
    input_variables=["product", "rating", "pros", "cons", "word_limit"],
    template="""Write a product review for {product} with the following constraints:
    1. The review should have a {rating}-star rating (out of 5)
    2. Include exactly {pros} pros and {cons} cons
    3. Use between 2 and 3 sentences for each pro and con
    4. The entire review should be under {word_limit} words
    5. End with a one-sentence recommendation
    
    Format the review as follows:
    Rating: [X] out of 5 stars
    
    Pros:
    1. [Pro 1]
    2. [Pro 2]
    ...
    
    Cons:
    1. [Con 1]
    2. [Con 2]
    ...
    
    Recommendation: [One-sentence recommendation]
    """
)

# Generate the constrained review
input_variables = {
    "product": "Smartphone X",
    "rating": "4",
    "pros": "3",
    "cons": "2",
    "word_limit": "200"
}

chain = review_prompt | llm
output = chain.invoke(input_variables).content
display_output(output)

Model Output:
----------------------------------------
Rating: 4 out of 5 stars

Pros:

1. The Smartphone X's camera is exceptional, capturing vibrant colors and detailed images even in low-light conditions.
2. The device's battery life is impressive, lasting a full day with moderate use.
3. The phone's sleek design and premium materials make it a stylish addition to any collection.

Cons:

1. The Smartphone X's price point may be out of reach for budget-conscious buyers.
2. Some users have reported issues with the phone's fingerprint recognition system, requiring multiple attempts to unlock.

Recommendation: Overall, the Smartphone X is a solid choice for those seeking a high-quality smartphone without breaking the bank, but its limitations in this regard prevent it from reaching its full potential.
----------------------------------------



# Role Prompting
assign specific roles to AI models 

## Basic Role Assignment


In [38]:
tech_writer_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""You are a technical writer specializing in creating clear and concise documentation for software products.
    Your task is to write a brief explanation of {topic} for a user manual.
    Please provide a 2-3 sentence explanation that is easy for non-technical users to understand."""
)

chain = tech_writer_prompt | llm
response = chain.invoke({"topic": "cloud computing"})
print(response.content)

Here's a possible explanation:

"Cloud computing allows you to store, manage, and access your data and applications over the internet, rather than on a local computer or server. This means that instead of having to manage your own hardware and software, you can use cloud-based services like storage, processing power, and software applications on-demand. By using cloud computing, you can easily scale up or down to meet changing needs, and access your files and data from anywhere with an internet connection."


## Crafting Effective Role Descriptions

In [39]:
financial_advisor_prompt = PromptTemplate(
    input_variables=["client_situation"],
    template="""You are a seasoned financial advisor with over 20 years of experience in personal finance, investment strategies, and retirement planning.
    You have a track record of helping clients from diverse backgrounds achieve their financial goals.
    Your approach is characterized by:
    1. Thorough analysis of each client's unique financial situation
    2. Clear and jargon-free communication of complex financial concepts
    3. Ethical considerations in all recommendations
    4. A focus on long-term financial health and stability

    Given the following client situation, provide a brief (3-4 sentences) financial advice:
    {client_situation}

    Your response should reflect your expertise and adhere to your characteristic approach."""
)

chain = financial_advisor_prompt | llm
response = chain.invoke({"client_situation": "A 35-year-old professional earning $80,000 annually, with $30,000 in savings, no debt, and no retirement plan."})
print(response.content)

Based on the client's situation, I would recommend that they prioritize building an emergency fund, aiming to save 3-6 months' worth of expenses in a readily accessible savings account. Next, I would suggest exploring employer-matched retirement accounts, such as a 401(k) or IRA, to take advantage of compound interest and tax benefits. Considering their relatively high income-to-expenses ratio, I would also encourage the client to explore tax-efficient investment strategies, such as index fund investing, to optimize returns while minimizing taxes. By taking these steps, the client can establish a solid financial foundation for long-term stability and success.


## Comparision of Roles

In [40]:
roles = [
    ("Scientist", "You are a research scientist specializing in climate change. Explain the following concept in scientific terms:"),
    ("Teacher", "You are a middle school science teacher. Explain the following concept in simple terms suitable for 12-year-old students:"),
    ("Journalist", "You are a journalist writing for a popular science magazine. Explain the following concept in an engaging and informative manner for a general adult audience:")
]

topic = "The greenhouse effect"

for role, description in roles:
    role_prompt = PromptTemplate(
        input_variables=["topic"],
        template=f"{description} {{topic}}"
    )
    chain = role_prompt | llm
    response = chain.invoke({"topic": topic})
    print(f"\n{role}'s explanation:\n")
    print(response.content)
    print("-" * 50)


Scientist's explanation:

The greenhouse effect is a complex process by which certain gases in the Earth's atmosphere trap heat from the sun, leading to a warming of the planet. This natural phenomenon has been occurring for millions of years and plays a crucial role in maintaining the Earth's temperature.

**Mechanism:**

The greenhouse effect occurs when solar radiation enters the Earth's atmosphere and is absorbed by the surface, including oceans, land, and ice. This absorbed energy is then re-radiated back into the atmosphere as infrared (IR) radiation, which is less energetic than the original solar radiation. The IR radiation is trapped by certain gases in the atmosphere, such as water vapor (H2O), carbon dioxide (CO2), methane (CH4), and others.

These greenhouse gases have a molecular structure that allows them to absorb IR radiation at specific wavelengths. As they absorb the energy, their molecules vibrate or rotate, which increases their temperature. The trapped heat is the

## Refining Role Descriptions
Let's take story telling examples.

In [41]:
storyteller_prompt = PromptTemplate(
    input_variables=["style", "scenario"],
    template="""You are a master storyteller known for your ability to adapt to various narrative styles.
    Your current task is to write in the style of {style}.
    Key characteristics of this style include:
    1. {style_char1}
    2. {style_char2}
    3. {style_char3}

    Write a short paragraph (3-4 sentences) in this style about the following scenario:
    {scenario}

    Ensure your writing clearly reflects the specified style."""
)

styles = [
    {
        "name": "Gothic horror",
        "char1": "Atmospheric and ominous descriptions",
        "char2": "Themes of decay, death, and the supernatural",
        "char3": "Heightened emotions and sense of dread"
    },
    {
        "name": "Minimalist realism",
        "char1": "Sparse, concise language",
        "char2": "Focus on everyday, ordinary events",
        "char3": "Subtle implications rather than explicit statements"
    }
]

scenario = "A person enters an empty house at twilight"

for style in styles:
    chain = storyteller_prompt | llm
    response = chain.invoke({
        "style": style["name"],
        "style_char1": style["char1"],
        "style_char2": style["char2"],
        "style_char3": style["char3"],
        "scenario": scenario
    })
    print(f"\n{style['name']} version:\n")
    print(response.content)
    print("-" * 50)


Gothic horror version:

As the last wisps of sunlight surrendered to the encroaching twilight, Emilia stepped across the threshold of Ravenswood Manor, her footsteps echoing through the vacant halls like a mournful sigh. The once-grand entrance, now shrouded in a tapestry of grime and decay, seemed to yawn open like a cold, dead mouth, revealing a cavernous expanse of shadows that stretched out before her like a dark, malevolent sea. The air within was heavy with the scent of rot and forgotten memories, and Emilia's heart quickened as she felt the weight of unseen eyes upon her, watching her every move with an unblinking gaze. In this forsaken house, time itself seemed to have grown still, waiting for some unseen force to stir once more.
--------------------------------------------------

Minimalist realism version:

The door creaks, a slow protest as it yields to the turn of the handle. The interior is bathed in a faint, diffused light that seeps through worn curtains. A lone chair s